# LSTM-arithmetic

## Dataset
- [Arithmetic dataset](https://drive.google.com/file/d/1cMuL3hF9jefka9RyF4gEBIGGeFGZYHE-/view?usp=sharing)

In [2]:
# ! pip install seaborn
# ! pip install opencc
# ! pip install -U scikit-learn

import numpy as np
import pandas as pd
import torch
import torch.nn
import torch.nn.utils.rnn
import torch.utils.data
# import matplotlib.pyplot as plt
# import seaborn as sns
# import opencc
import os
# from sklearn.model_selection import train_test_split

data_path = './data'

In [3]:
df_train = pd.read_csv(os.path.join(data_path, 'arithmetic_train.csv'))
df_eval = pd.read_csv(os.path.join(data_path, 'arithmetic_eval.csv'))
df_train.head()

,Unnamed: 0,src,tgt
0,2285313,14*(43+20)=,882
1,317061,(6+1)*5=,35
2,718770,13+32+29=,74
3,170195,31*(3-11)=,-248
4,2581417,24*49+1=,1177


In [4]:
# transform the input data to string
df_train['tgt'] = df_train['tgt'].apply(lambda x: str(x))
df_train['src'] = df_train['src'].add(df_train['tgt'])
df_train['len'] = df_train['src'].apply(lambda x: len(x))

df_eval['tgt'] = df_eval['tgt'].apply(lambda x: str(x))
df_eval['src'] = df_eval['src'].add(df_eval['tgt'])
df_eval['len'] = df_eval['src'].apply(lambda x: len(x))

# Build Dictionary
 - The model cannot perform calculations directly with plain text.
 - Convert all text (numbers/symbols) into numerical representations.
 - Special tokens
    - '&lt;pad&gt;'
        - Each sentence within a batch may have different lengths.
        - The length is padded with '&lt;pad&gt;' to match the longest sentence in the batch.
    - '&lt;eos&gt;'
        - Specifies the end of the generated sequence.
        - Without '&lt;eos&gt;', the model will not know when to stop generating.

In [5]:
char_to_id = {}
id_to_char = {}

# write your code here
# Build a dictionary and give every token in the train dataset an id
# The dictionary should contain <eos> and <pad>
# char_to_id is to conver charactors to ids, while id_to_char is the opposite
char_to_id = {char: i for i, char in enumerate(set(''.join(df_train['src'])))}
for char, id in char_to_id.items():
    char_to_id[char] = id + 2
char_to_id['<eos>'] = 1
char_to_id['<pad>'] = 0
id_to_char = {i: char for char, i in char_to_id.items()}

vocab_size = len(char_to_id)
print('Vocab size {}'.format(vocab_size))
print(char_to_id)

Vocab size 18
{'0': 2, '5': 3, ')': 4, '9': 5, '1': 6, '(': 7, '4': 8, '=': 9, '8': 10, '+': 11, '-': 12, '2': 13, '7': 14, '*': 15, '3': 16, '6': 17, '<eos>': 1, '<pad>': 0}


# Data Preprocessing
 - The data is processed into the format required for the model's input and output.
 - Example: 1+2-3=0
     - Model input: 1 + 2 - 3 = 0
     - Model output: / / / / / 0 &lt;eos&gt;  (the '/' can be replaced with &lt;pad&gt;)
     - The key for the model's output is that the model does not need to predict the next character of the previous part. What matters is that once the model sees '=', it should start generating the answer, which is '0'. After generating the answer, it should also generate&lt;eos&gt;


In [7]:
# Write your code here
df_train['char_id_list'] = df_train['src'].apply(lambda x: [char_to_id[c] for c in x]+[char_to_id['<eos>']])
df_train['label_id_list'] = df_train['char_id_list'].apply(lambda x: [char_to_id["<pad>"]]*x.index(char_to_id["="]) + x[x.index(char_to_id["="])+1:])

df_eval['char_id_list'] = df_eval['src'].apply(lambda x: [char_to_id[c] for c in x]+[char_to_id['<eos>']])
df_eval['label_id_list'] = df_eval['char_id_list'].apply(lambda x: [char_to_id["<pad>"]]*x.index(char_to_id["="]) + x[x.index(char_to_id["="])+1:])


df_train.head()

,Unnamed: 0,src,tgt,len,char_id_list,label_id_list
0,2285313,14*(43+20)=882,882,14,"[6, 8, 15, 7, 8, 16, 11, 13, 2, 4, 9, 10, 10, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 13, 1]"
1,317061,(6+1)*5=35,35,10,"[7, 17, 11, 6, 4, 15, 3, 9, 16, 3, 1]","[0, 0, 0, 0, 0, 0, 0, 16, 3, 1]"
2,718770,13+32+29=74,74,11,"[6, 16, 11, 16, 13, 11, 13, 5, 9, 14, 8, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 14, 8, 1]"
3,170195,31*(3-11)=-248,-248,14,"[16, 6, 15, 7, 16, 12, 6, 6, 4, 9, 12, 13, 8, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 13, 8, 10, 1]"
4,2581417,24*49+1=1177,1177,12,"[13, 8, 15, 8, 5, 11, 6, 9, 6, 6, 14, 14, 1]","[0, 0, 0, 0, 0, 0, 0, 6, 6, 14, 14, 1]"


# Hyper Parameters

|Hyperparameter|Meaning|Value|
|-|-|-|
|`batch_size`|Number of data samples in a single batch|64|
|`epochs`|Total number of epochs to train|10|
|`embed_dim`|Dimension of the word embeddings|256|
|`hidden_dim`|Dimension of the hidden state in each timestep of the LSTM|256|
|`lr`|Learning Rate|0.001|
|`grad_clip`|To prevent gradient explosion in RNNs, restrict the gradient range|1|

In [8]:
batch_size = 64
epochs = 2
embed_dim = 256
hidden_dim = 256
lr = 0.001
grad_clip = 1

# Data Batching
- Use `torch.utils.data.Dataset` to create a data generation tool called  `dataset`.
- The, use `torch.utils.data.DataLoader` to randomly sample from the `dataset` and group the samples into batches.

In [32]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, sequences: pd.DataFrame):
        self.sequences = sequences
    
    def __len__(self):
        return self.sequences.shape[0]
    
    def __getitem__(self, index):
        # Extract the input data x and the ground truth y from the data
        x = self.sequences['char_id_list'].iloc[index][:-1]
        y = self.sequences['label_id_list'].iloc[index]
        return x, y

# collate function, used to build dataloader
def collate_fn(batch):
    batch_x = [torch.tensor(data[0]) for data in batch]
    batch_y = [torch.tensor(data[1]) for data in batch]
    batch_x_lens = torch.LongTensor([len(x) for x in batch_x])
    batch_y_lens = torch.LongTensor([len(y) for y in batch_y])
    
    # Pad the input sequence
    pad_batch_x = torch.nn.utils.rnn.pad_sequence(batch_x,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    pad_batch_y = torch.nn.utils.rnn.pad_sequence(batch_y,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    return pad_batch_x, pad_batch_y, batch_x_lens, batch_y_lens

In [33]:
ds_train = Dataset(df_train[['char_id_list', 'label_id_list']])
ds_eval = Dataset(df_eval[['char_id_list', 'label_id_list']])

In [34]:
# Build dataloader of train set and eval set, collate_fn is the collate function
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
dl_eval = torch.utils.data.DataLoader(ds_eval, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Model Design

## Execution Flow
1. Convert all characters in the sentence into embeddings.
2. Pass the embeddings through an LSTM sequentially.
3. The output of the LSTM is passed into another LSTM, and additional layers can be added.
4. The output from all time steps of the final LSTM is passed through a Fully Connected layer.
5. The character corresponding to the maximum value across all output dimensions is selected as the next character.

## Loss Function
Since this is a classification task, Cross Entropy is used as the loss function.

## Gradient Update
Adam algorithm is used for gradient updates.

In [28]:
class CharRNN(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharRNN, self).__init__()
        
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=embed_dim,
                                            padding_idx=char_to_id['<pad>'])
        
        self.rnn_layer1 = torch.nn.LSTM(input_size=embed_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.rnn_layer2 = torch.nn.LSTM(input_size=hidden_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.linear = torch.nn.Sequential(torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=hidden_dim),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=vocab_size))
        
    def forward(self, batch_x, batch_x_lens):
        return self.encoder(batch_x, batch_x_lens)
    
    # The forward pass of the model
    def encoder(self, batch_x, batch_x_lens):
        batch_x = self.embedding(batch_x)
        
        batch_x = torch.nn.utils.rnn.pack_padded_sequence(batch_x,
                                                          batch_x_lens,
                                                          batch_first=True,
                                                          enforce_sorted=False)
        
        batch_x, _ = self.rnn_layer1(batch_x)
        batch_x, _ = self.rnn_layer2(batch_x)
        
        batch_x, _ = torch.nn.utils.rnn.pad_packed_sequence(batch_x,
                                                            batch_first=True)
        
        batch_x = self.linear(batch_x)
        
        return batch_x
    
    def generator(self, start_char, max_len=200):
        
        char_list = [char_to_id[c] for c in start_char]
        
        next_char = None
        
        while len(char_list) < max_len: 
            # Write your code here 
            # Pack the char_list to tensor
            # Input the tensor to the embedding layer, LSTM layers, linear respectively
            # Obtain the next token prediction y
            y =  self.embedding(torch.tensor(char_list).unsqueeze(0))
            y, _ = self.rnn_layer1(y)
            y, _ = self.rnn_layer2(y)
            y = self.linear(y)
            
            # Use argmax function to get the next token prediction
            next_char = torch.argmax(y[0, -1]).item()
            
            if next_char == char_to_id['<eos>']:
                break
            
            char_list.append(next_char)
            
        return [id_to_char[ch_id] for ch_id in char_list]

In [29]:
torch.manual_seed(2)


device = "cuda:3"

model = CharRNN(vocab_size,
                embed_dim,
                hidden_dim)

In [30]:
# Write your code here. Cross-entropy loss function. The loss function should ignore <pad>
criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])
# Write your code here. Use Adam or AdamW for Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Training
1. The outer `for` loop controls the `epoch`
    1. The inner `for` loop uses `data_loader` to retrieve batches.
        1. Pass the batch to the `model` for training.
        2. Compare the predicted results `batch_pred_y` with the true labels `batch_y` using Cross Entropy to calculate the loss `loss`
        3. Use `loss.backward` to automatically compute the gradients.
        4. Use `torch.nn.utils.clip_grad_value_` to limit the gradient values between `-grad_clip` &lt; and &lt; `grad_clip`.
        5. Use `optimizer.step()` to update the model (backpropagation).
2.  After every `1000` batches, output the current loss to monitor whether it is converging.

In [35]:
from tqdm import tqdm
from copy import deepcopy
model = model.to(device)
model.train()
i = 0
for epoch in range(1, epochs+1):
    # The process bar
    bar = tqdm(dl_train, desc=f"Train epoch {epoch}")
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        # Write your code here
        # Clear the gradient
        optimizer.zero_grad()
    
        batch_pred_y = model(batch_x.to(device), batch_x_lens)
        
        # Write your code here
        # Input the prediction and ground truths to loss function
        # Back propagation
        loss = criterion(batch_pred_y.permute(0, 2, 1), batch_y.to(device))
        loss.backward()

        torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip) # gradient clipping

        # Write your code here
        # Optimize parameters in the model
        optimizer.step()

        i+=1
        if i%50==0:
            bar.set_postfix(loss = loss.item())
    
    # Evaluate your model
    bar = tqdm(dl_eval, desc=f"Validation epoch {epoch}")
    matched = 0
    total = 0
    with torch.inference_mode():
        for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
            # Write your code here. Input the batch_x to the model and generate the predictions
            predictions = model(batch_x.to(device), batch_x_lens)
            
            # Write your code here.
            # Check whether the prediction match the ground truths
            # Compute exact match (EM) on the eval dataset
            # EM = correct/total
            matched += torch.sum(torch.argmax(predictions, dim=2) == batch_y.to(device)).item()

        
    print(matched/total)

Train epoch 1:   0%|          | 0/37020 [00:00<?, ?it/s]

Train epoch 1:   4%|▍         | 1590/37020 [00:34<12:44, 46.33it/s, loss=1.03] 


KeyboardInterrupt: 

# Generation
Use `model.generator` and provide an initial character to automatically generate a sequence.

In [21]:
model = model.to("cpu")
print("".join(model.generator('1+1=')))

1+1=2
